In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, k, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior
    
    sync = PostponeScheduler()

    for i in 1:n
        x[i] ~ (x_prev + c) where { portal = rem(i, k) === 0 ? ScheduleOnPortal(sync) : EmptyPortal() }
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return sync, x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0

n = 10_000
k = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(; data, k, x0, P)
    n = length(data)
    
    _, (sync, x, y) = smoothing(n, k, x0, P);

    buffer    = Vector{Marginal}(undef, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    wait(sync)
    
    update!(y, data)
    
    wait(sync)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [5]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 
  memory estimate:  358.34 MiB
  allocs estimate:  6010542
  --------------
  minimum time:     696.772 ms (35.69% GC)
  median time:      804.851 ms (46.06% GC)
  mean time:        816.891 ms (45.90% GC)
  maximum time:     1.037 s (56.44% GC)
  --------------
  samples:          7
  evals/sample:     1